# Cvm Daily Funds Data

In [1]:
from catalog_api.infrastructure.CkanApi import CkanApi
import pandas as pd
from datetime import datetime

from catalog_api.services.CatalogDataService import CatalogDataService

catalog_data = CatalogDataService()
portal = CkanApi("https://dados.cvm.gov.br")
organization = "br_gov_cvm"
dataset = "fi-doc-inf_diario"
table_name = "inf_diario_fi"
fi_doc_inf_diario = portal.get_package(dataset)

In [ ]:
def process_resource(resource):
    url = resource.url
    filename = url.split("/")[-1]
    filename_prefix = filename.split(".")[0]
    partition_full = filename_prefix.split("_")[-1]
    year = partition_full[:4]
    month = partition_full[4:6]
    if not year.isnumeric() or not month.isnumeric():
        print(f"Invalid partition: {filename_prefix}")
        return
    partitions = {
        "year": year,
        "month": month,
    }
    bucket_last_updated = catalog_data.get_parquet_last_updated(
        organization, dataset, table_name, partitions
    ) or datetime(2000, 1, 1)
    resource_last_updated = datetime.fromisoformat(
        resource.created or resource.last_modified
    )
    if bucket_last_updated >= resource_last_updated:
        return
    df = pd.read_csv(url, sep=";", encoding="latin1", low_memory=False)
    df["DT_COMPTC"] = df["DT_COMPTC"].apply(
        lambda x: datetime.strptime(x, "%Y-%m-%d").date() if x else None
    )
    res = catalog_data.put_parquet_data(df, organization, dataset, table_name, partitions)
    print(res)

for resource in fi_doc_inf_diario.resources: # type: ignore
    process_resource(resource)

Invalid partition: meta_inf_diario_fi
{'ResponseMetadata': {'RequestId': 'TCT274HSHS9WCTBS', 'HostId': 'B1w2zPwfafheeo9vomPOdSyx5RCvM/aTYQYcDpJNREafWGZ+esDZt28TqlcfhgBkA3RmsqdqC+iEaDUW7F+v+YV7A33jYSJ0ZB71n0g0uXk=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'B1w2zPwfafheeo9vomPOdSyx5RCvM/aTYQYcDpJNREafWGZ+esDZt28TqlcfhgBkA3RmsqdqC+iEaDUW7F+v+YV7A33jYSJ0ZB71n0g0uXk=', 'x-amz-request-id': 'TCT274HSHS9WCTBS', 'date': 'Sat, 24 May 2025 03:39:43 GMT', 'x-amz-server-side-encryption': 'AES256', 'etag': '"6fad2caf750d8cc412c0476c03b32bfb"', 'x-amz-checksum-crc32': 'EW+ZoQ==', 'x-amz-checksum-type': 'FULL_OBJECT', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'ETag': '"6fad2caf750d8cc412c0476c03b32bfb"', 'ChecksumCRC32': 'EW+ZoQ==', 'ChecksumType': 'FULL_OBJECT', 'ServerSideEncryption': 'AES256'}


## Query the data

In [ ]:
from catalog_api.infrastructure.DuckDbParquet import DuckDbParquet


db = DuckDbParquet(
    bucket,
    "catalog-data/br_gov_cvm/fi-doc-inf_diario",
    "catalog-data/br_gov_cvm/fi-doc-inf_diario/_duckdb",
)
db.query("SELECT * FROM 'data.parquet' WHERE DT_COMPTC IS NOT NULL LIMIT 10")

IOException: IO Error: Cannot open file "/home/rportela/github/rportela/py-api-catalog/notebooks/catalog-data/br_gov_cvm/fi-doc-inf_diario/_duckdb": No such file or directory